In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from requests.exceptions import HTTPError
from random import randint
import time

# Get the list of words

In [56]:
df = pd.read_csv("word_list.csv")
df = df.iloc[10:25]
df.head(20)

,words,levels,def_urls,types,sound_urls,sound_files
10,abroad,A2,https://www.oxfordlearnersdictionaries.com/def...,(adverb),https://www.oxfordlearnersdictionaries.com/med...,abroad__us_4.mp3
11,absence,C1,https://www.oxfordlearnersdictionaries.com/def...,(noun),https://www.oxfordlearnersdictionaries.com/med...,absence__us_1.mp3
12,absent,C1,https://www.oxfordlearnersdictionaries.com/def...,(adjective),https://www.oxfordlearnersdictionaries.com/med...,absent__us_1.mp3
13,absolute,B2,https://www.oxfordlearnersdictionaries.com/def...,(adjective),https://www.oxfordlearnersdictionaries.com/med...,absolute__us_1_rr.mp3
14,absolutely,B1,https://www.oxfordlearnersdictionaries.com/def...,(adverb),https://www.oxfordlearnersdictionaries.com/med...,absolutely__us_1_rr.mp3
15,absorb,B2,https://www.oxfordlearnersdictionaries.com/def...,(verb),https://www.oxfordlearnersdictionaries.com/med...,absorb__us_2_rr.mp3
16,abstract,B2,https://www.oxfordlearnersdictionaries.com/def...,(adjective),https://www.oxfordlearnersdictionaries.com/med...,abstract__us_2_rr.mp3
17,absurd,C1,https://www.oxfordlearnersdictionaries.com/def...,(adjective),https://www.oxfordlearnersdictionaries.com/med...,absurd__us_1_rr.mp3
18,abundance,C1,https://www.oxfordlearnersdictionaries.com/def...,(noun),https://www.oxfordlearnersdictionaries.com/med...,abundance__us_1.mp3
19,abuse,C1,https://www.oxfordlearnersdictionaries.com/def...,(noun),https://www.oxfordlearnersdictionaries.com/med...,abuse__us_3.mp3


# Scraping of all the translation pages

In [57]:
BASE_URL = "https://www.larousse.fr/dictionnaires/anglais-francais/"
user_agents = [
        {'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.131 Safari/537.36"},
        {'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.107 Safari/537.36"},
        {'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:90.0) Gecko/20100101 Firefox/90.0"},
        {'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.164 Safari/537.36"},
        {'User-Agent': "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.131 Safari/537.36"},
        {'User-Agent': "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.107 Safari/537.36"},
        {'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"},
        {'User-Agent': "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.1.2 Safari/605.1.15"},
        {'User-Agent': "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.1.1 Safari/605.1.15"},
        {'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:91.0) Gecko/20100101 Firefox/91.0"},
        {'User-Agent': "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.111 Safari/537.36"},
        {'User-Agent': "Mozilla/5.0 (X11; CrOS x86_64 8172.45.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.64 Safari/537.36"},
        {'User-Agent': "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:15.0) Gecko/20100101 Firefox/15.0.1"},
        {'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.135 Safari/537.36 Edge/12.246"}
    ]


def scrape_webpage(req_url):
    """Get a webpage with the requests module & return the scrape translation of a word"""
    try:
        req_response = requests.get(req_url, headers=user_agents[randint(0, len(user_agents))])
        req_response.raise_for_status()
        s = BeautifulSoup(req_response.content, 'html.parser')
        return list(set([elt.text for elt in s.find_all("a", class_="lienarticle2")]))
    except HTTPError as http_err:
        print(f'HTTP error occurred: {http_err} for URL {req_url}')
        return False
    except Exception as err:
        print(f'Other error occurred: {err} for URL {req_url}')
        return False
        

In [58]:
for w in df['words'].values:
    time.sleep(5)
    translation = ""
    try:
        translation = scrape_webpage(BASE_URL + w + "/")
    except:
        pass
    if translation is not False:
        if len(translation) > 4: translation = translation[:4]
        if len(translation) > 1:
            translation = '<b>' + translation[0] + '</b> <i>(' + ", ".join([t for t in translation[1:]]) + ')</i>'
        df.loc[df[df['words'] == w].index, "translations"] = translation

    
df[['words', 'translations']].head(20)

Other error occurred: list index out of range for URL https://www.larousse.fr/dictionnaires/anglais-francais/absence/
Other error occurred: list index out of range for URL https://www.larousse.fr/dictionnaires/anglais-francais/academic/
Other error occurred: list index out of range for URL https://www.larousse.fr/dictionnaires/anglais-francais/accelerate/


,words,translations
10,abroad,"<b>partout</b> <i>(de tous côtés, au loin, à l..."
11,absence,NaN
12,absent,<b>absent</b> <i>(distrait)</i>
13,absolute,"<b>définitif</b> <i>(formel, anhydre, indiscut..."
14,absolutely,"<b>absolument</b> <i>(formellement, vraiment)</i>"
15,absorb,"<b>résorber</b> <i>(assimiler, amortir, absorb..."
16,abstract,"<b>dérober</b> <i>(abstrait, abstraire, extrai..."
17,absurd,"<b>ridicule</b> <i>(insensé, absurde)</i>"
18,abundance,<b>profusion</b> <i>(abondance)</i>
19,abuse,"<b>abus</b> <i>(injurier, malmener, maltraiter..."


In [60]:
df[df['translations'].isnull()]['words']

11       absence
24    accelerate
Name: words, dtype: object